<a href="https://colab.research.google.com/github/Dhaneshkp/NLP/blob/main/FEakNEws_LSTM_withBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fake News Classifier Using Bidirectional LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import pandas as pd
#!pip install kaggle



In [60]:
!pip install gensim==4.3.1
!pip install transformers
!pip install tensorflow-gpu

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
#import gensim.downloader as api
#wv = api.load('word2vec-google-news-300')

In [3]:
from transformers import BertTokenizer, TFBertModel
import torch

In [4]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = TFBertModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

!kaggle competitions download -c fake-news

In [16]:
import os
os.getcwd()

'/content'

In [18]:
df=pd.read_csv(r'train.csv')

In [19]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [20]:
###Drop Nan Values
df=df.dropna()


In [21]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [22]:
## Get the Dependent features
y=df['label']

In [23]:
y.value_counts()

label
0    10361
1     7924
Name: count, dtype: int64

In [24]:
X.shape

(18285, 4)

In [25]:
y.shape

(18285,)

In [26]:
import tensorflow as tf

In [27]:
tf.__version__

'2.15.0'

In [28]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [29]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [30]:
messages=X.copy()

In [31]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [32]:
messages.reset_index(inplace=True)

In [33]:
import nltk
import re
from nltk.corpus import stopwords

In [34]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [35]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')  # Download WordNet (lexical database)
nltk.download('omw-1.4')
nltk.download('punkt')  # For sentence tokenization
nltk.download('averaged_perceptron_tagger')  # For POS tagging
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [36]:

lemmatizer = WordNetLemmatizer()

#take mapping
def get_wordnet_pos(treebank_tag):
    """Maps treebank POS tags to WordNet POS tags."""
    if treebank_tag.startswith('J'):
        return 'a'  # Adjective
    elif treebank_tag.startswith('V'):
        return 'v'  # Verb
    elif treebank_tag.startswith('N'):
        return 'n'  # Noun
    elif treebank_tag.startswith('R'):
        return 'r'  # Adverb
    else:
        return ''

def lemmatiser_word(sentence):
    lemma =[]
    tokens = word_tokenize(sentence)  # Tokenize the sentence into words
    tagged_words = pos_tag(tokens)    # Tag each word with its POS
    for word,tag in tagged_words:
        if not word in stopwords.words('english'):
            pos_tag_simple=get_wordnet_pos(tag)
            if pos_tag_simple:
                lemma.append(lemmatizer.lemmatize(word, pos=pos_tag_simple))
            else:
                lemma.append(lemmatizer.lemmatize(word))

    return " ".join(lemma)

In [37]:
print(lemmatiser_word(messages['title'][20]))

Abortion Pill Orders Rise 7 Latin American Nations Zika Alert - The New York Times


In [38]:
### Dataset Preprocessingfrom nltk.stem import WordNetLemmatizer


from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    #print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    #review = review.split()

    #review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    #review = ' '.join(review)
    review = lemmatiser_word(review)

    corpus.append(review)

In [39]:
corpus

['house dem aide even see comey letter jason chaffetz tweet',
 'flynn hillary clinton big woman campus breitbart',
 'truth might get fire',
 'civilian kill single u airstrike identify',
 'iranian woman jail fictional unpublished story woman stone death adultery',
 'jackie mason hollywood would love trump bomb north korea lack trans bathroom exclusive video breitbart',
 'beno hamon win french socialist party presidential nomination new york time',
 'back channel plan ukraine russia courtesy trump associate new york time',
 'obama organize action partner soros link indivisible disrupt trump agenda',
 'bbc comedy sketch real housewife isi cause outrage',
 'russian researcher discover secret nazi military base treasure hunter arctic photo',
 'u official see link trump russia',
 'yes paid government troll social medium blog forum website',
 'major league soccer argentine find home success new york time',
 'well fargo chief abruptly step new york time',
 'anonymous donor pay million release 

#onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr=encoding_tfidf(corpus)
pad_length=20

array=[]
for i in range(len(corpus)):
    array.append(drop_zeroes(onehot_repr[i]))
    
pad_sequences(array,padding='pre',maxlen=pad_length)    

array.shape

In [40]:
def drop_zeroes(array):
    pad_length = 20
    return_array=[]

    for i in array:
        if i == 0 :
            pass
        else :
            return_array.append(i)
    return return_array



In [41]:
print(drop_zeroes([0,0,0,0,0,0,2, 2 , 3 ,6,0,0,0,0,0,0,0]))

[2, 2, 3, 6]


In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
feature_names=[]
def encoding_tfidf(corpus):
    # Vectorize the text data using TF-IDF
    global  voc_size
    global feature_names
    vectorizer = TfidfVectorizer()
    data = vectorizer.fit_transform(corpus)
    # Get the feature names (vocabulary)
    feature_names = vectorizer.get_feature_names_out()
    print()
    word_to_index = {word: index for index, word in enumerate(feature_names)}
    voc_size =len(feature_names)
    #Numerical Representation (Using TF-IDF Values)
    sequences = []
    for row in data.toarray():
        sentence_indices = [word_to_index[feature_names[i]] for i in np.nonzero(row)[0]]
        sequences.append(sentence_indices)
    return sequences


vectorizer = TfidfVectorizer(max_features= 300)
data = vectorizer.fit_transform(corpus)

#for j in range(data.toarray().shape[0]):
#    for i in range(data.toarray().shape[1]):
#        if data.toarray()[j][i] != 0 :
#            print(f' message {j}  word {i} = {data.toarray()[j][i]}')

In [43]:
#from gensim.models import Word2Vec


In [44]:
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    return [word for word in tokens if word not in stopwords.words('english') and word.isalnum()]

In [45]:
corpus_wv = [preprocess_text(message) for message in corpus]

In [46]:
#corpus_wv

In [47]:

#model = Word2Vec(sentences=corpus_wv, vector_size=300, window=5, min_count=1, workers=4)

In [48]:

#embedding_matrix = np.zeros((len(model.wv), 300))
#for i, word in enumerate(model.wv.key_to_index):
#    embedding_matrix[i] = model.wv[word]

In [49]:
#embedding_matrix.shape

In [50]:
#vocab=model.wv.key_to_index
#voc_size=embedding_matrix.shape[0]

In [51]:
#word_to_index = {word: index for index, word in enumerate(vocab)}

In [52]:
#word_to_index

In [53]:


def text_to_sequence(text):
    return [word_to_index[word] for word in text]

#sequences = [text_to_sequence(message) for message in corpus_wv]

In [54]:
#sequences

In [55]:
#max_length = 20
#padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
#pd.DataFrame(padded_sequences,columns=[f'column{column}' for column in range(1,21)])

In [56]:
#padded_sequences.shape

In [61]:
def encode_sentences(sentences):
  input_ids = []
  attention_masks = []
  for sentence in sentences:
    encoding = tokenizer.encode_plus(
      sentence,
      add_special_tokens=True,
      max_length=64,
      padding='max_length',
      truncation=True,
      return_attention_mask=True,
      return_tensors='tf'
    )
    input_ids.append(encoding['input_ids'])
    attention_masks.append(encoding['attention_mask'])
  return tf.concat(input_ids, axis=0), tf.concat(attention_masks, axis=0)

# Example usage:
input_ids, attention_masks = encode_sentences(corpus)


In [70]:
import torch
if torch.cuda.is_available():
 device = torch.device('cuda')


In [76]:
# Training loop (example)
num_epochs = 5
batch_size = 32

from torch.utils.data import DataLoader, TensorDataset

# Assuming 'input_ids', 'attention_masks', and 'labels' are already defined as tensors
dataset = TensorDataset(input_ids, attention_masks)  # Create a TensorDataset
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)  # Create a DataLoader

for epoch in range(num_epochs):
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_masks = batch['attention_masks'].to(device)
        optimizer.zero_grad()
        outputs = bert_model(input_ids, attention_masks).to(device)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()




AttributeError: EagerTensor object has no attribute 'size'. 
        If you are looking for numpy-related methods, please run the following:
        tf.experimental.numpy.experimental_enable_numpy_behavior()
      

### Embedding Representation

In [ ]:
embedding_layer = Embedding(input_dim=voc_size,
                            output_dim=300,
                            weights=[embedding_matrix],
                            input_length=max_length,
                            trainable=False)

In [ ]:
## Creating model
embedding_vector_features=40
model=Sequential()
model1.add(embedding_layer)
#model.add(Embedding(voc_size,embedding_vector_features,input_length=max_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

In [ ]:
## Creating model
sent_length=max_length
embedding_vector_features=40
model1=Sequential()
#model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(embedding_layer)
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.5))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

In [ ]:
len(padded_sequences),y.shape

In [ ]:
data=embedding_matrix

In [ ]:
import numpy as np
#X_final=padded_sequences
X_final= padded_sequences
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [ ]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

### Performance Metrics And Accuracy

In [ ]:

y_pred=model1.predict(X_test)
y_pred1=[1 if i>=0.5 else 0 for i in y_pred ]

In [ ]:
#y_pred1

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,y_pred1)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))